In [2]:
import pandas as pd
from data import Urg2022_pub, UrgP2022, Urg2022_priv_luc, Urg2022_priv_non_luc

In [4]:
###---Productivité des médecins---

#Nombre de passages aux urgences par médecin en 2022 selon le secteur 

#Extraction du nombre d'ETP de médecins
Medecins2022 = UrgP2022[UrgP2022["PERSO"].isin(["M1000","M2000","M3020"])]
ETP_Medecins2022 = Medecins2022.groupby("FI",as_index=False)[["ETPSAL","EFFLIB"]].sum()
ETP_Medecins2022["tot"] = ETP_Medecins2022["EFFLIB"] + ETP_Medecins2022["ETPSAL"]


In [ ]:

#Rassemble les médecins ETP avec le nombre de passages aux urgences selon le type d'établissement
#et calcule le ratio passages aux urgences/ETP médecins
Passages_pub = Urg2022_pub[["FI","PASSU"]]
Passages_pub = Passages_pub.merge(ETP_Medecins2022, left_on="FI", right_on="FI", how="left")
Passages_pub.dropna(inplace = True)
Passages_pub["ratio"] = Passages_pub["PASSU"]/Passages_pub["tot"]
Passages_pub["ratio"].describe()

count    6.700000e+01
mean     1.254575e+05
std      3.869213e+05
min      0.000000e+00
25%      1.877809e+04
50%      3.991238e+04
75%      6.958750e+04
max      2.660800e+06
Name: ratio, dtype: float64

In [16]:
#privé lucratif
Passages_priv_luc = Urg2022_priv_luc[["FI","PASSU"]]
Passages_priv_luc = Passages_priv_luc.merge(ETP_Medecins2022, left_on="FI", right_on="FI", how="left")
Passages_priv_luc.dropna(inplace=True)
Passages_priv_luc["ratio"] = Passages_priv_luc["PASSU"]/Passages_priv_luc["tot"]
Passages_priv_luc["ratio"].describe()

count        1.0
mean     17863.0
std          NaN
min      17863.0
25%      17863.0
50%      17863.0
75%      17863.0
max      17863.0
Name: ratio, dtype: float64

In [17]:
#privé non lucratif
Passages_priv_nonluc = Urg2022_priv_non_luc[["FI","PASSU"]]
Passages_priv_nonluc = Passages_priv_nonluc.merge(ETP_Medecins2022, left_on="FI", right_on="FI", how="left")
Passages_priv_nonluc.dropna(inplace=True)
Passages_priv_nonluc["ratio"] = Passages_priv_nonluc["PASSU"]/Passages_priv_nonluc["tot"]
Passages_priv_nonluc["ratio"].describe()

count         2.000000
mean     355291.666667
std      325860.732019
min      124873.333333
25%      240082.500000
50%      355291.666667
75%      470500.833333
max      585710.000000
Name: ratio, dtype: float64